In [67]:
import spacy
from meme_entity_detection.dataset.data_module import DataModule
from pathlib import Path
from IPython.display import clear_output
import pandas as pd
from spacy.cli import download
import base64
from openai import OpenAI
from tqdm import tqdm
import json


# Download the large English model if it is not already installed
try:
    nlp = spacy.load("en_core_web_lg")
except OSError:
    download("en_core_web_lg")
    
    
nlp = spacy.load("en_core_web_lg")


In [2]:
   
def count_known_words(text, nlp=nlp):    
    # Process the text
    doc = nlp(text)
    
    # Count known words
    known_words = sum(1 for token in doc if not token.is_oov)
    total_words = len(doc)
    
    return known_words, total_words


In [54]:
data_dir = Path("../../../data/HVVMemes/")
dataset = DataModule(data_dir, balance_train_dataset=False)
dataset.setup("_")
clear_output(wait=False)

train_df = dataset.train_dataset.data_df
validation_df = dataset.validation_dataset.data_df
test_df  = dataset.test_dataset.data_df

# Concatenating all dataframes to get a combined view
combined_df = pd.concat([train_df.assign(dataset='train'), 
                         validation_df.assign(dataset='validation'), 
                         test_df.assign(dataset='test')])[["sentence", "original", "dataset", "image"]].drop_duplicates()

combined_df["image_path"] = str(data_dir) + "/images/" + combined_df["image"]

In [4]:
word_counter = combined_df["sentence"].apply(count_known_words)
combined_df["correct words"] = word_counter.apply(lambda x: x[0])
combined_df["all words"] = word_counter.apply(lambda x: x[1])

correct_words_ratio = (combined_df["correct words"].sum() / combined_df["all words"].sum())
print(f"Correct words ration of existing data:  {round(correct_words_ratio*100, 2)} %")

Correct words ration of existing data:  92.87 %


In [5]:
#organization = input("Please insert your organization id:"),
api_key = input("Please insert your api key:")

clear_output()
client = OpenAI(
#  organization=organization,
  api_key=api_key
)



In [69]:
def send_api_call(image: base64):
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "You are an OCR Parser. Please give the full text shown in the image. Do not give anything else. If there is no text, do not answer anything."
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{image}"
            }
          
          },
        ],
      }
    ],
    max_tokens=300,
  )
      
  return response.choices[0].message.content

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [87]:
data_dir.parent / f'ocr_data_openai.json'

PosixPath('../../../data/ocr_data_openai.json')

In [70]:
i = 0
ocr_data = {}

for sentence, image, image_path in tqdm(zip(combined_df["sentence"].tolist(), combined_df["image"].tolist(), combined_df["image_path"].tolist())):
    print(sentence, image, image_path)
    
    i += 1
    if i == 30:
        input("Waiting for further input: ")
    
    encoded_image = encode_image(image_path)
    content = send_api_call(encoded_image)
 
    ocr_data[image] = content
    
    str(data_dir.parent)
    
        # Save the OCR data after each iteration
    with open(data_dir.parent / f'ocr_data_openai.json', 'w') as f:
        json.dump(ocr_data, f)

6716it [00:00, 155775.84it/s]

realtor: can you see yourself being quarantined here  covid_memes_4999.png ../../../data/HVVMemes/images/covid_memes_4999.png
when you connect your charger on 1% chárge but remóve it on f5% when you die but your cats stil wanna meow at you at 3am for food your phone: you won't let me et me de  covid_memes_4355.png ../../../data/HVVMemes/images/covid_memes_4355.png
i tyt tyt reacts: first presidential debate ne ms  memes_4435.png ../../../data/HVVMemes/images/memes_4435.png
the democratic party is socialist. i am the democratic party.  memes_1692.png ../../../data/HVVMemes/images/memes_1692.png
every disaster movie starts with the government ignoring a scientist tw  covid_memes_1094.png ../../../data/HVVMemes/images/covid_memes_1094.png
leaked footage of japan preparing for www3  covid_memes_805.png ../../../data/HVVMemes/images/covid_memes_805.png
ifi only had a you know, the thing  memes_6901.png ../../../data/HVVMemes/images/memes_6901.png
democrats propose stopping coyotes by distra